In [ ]:
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests, json, re, datetime, os, uuid
from urllib.parse import urljoin

In [ ]:
from icalendar import Calendar, Event, vText
import icalendar
import pytz

In [ ]:
def get_field_address(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    ul = soup.find_all("ul", {"class": "abssa_list"})
    if len(ul) == 0:
        return "Not found", "Not found"
    locality = re.search("Localité:(.+)", ul[3].text).group(1).strip()
    street = re.search("Rue:(.+)", ul[3].text).group(1).strip()
    name = re.search("Situation:(.+)", ul[3].text).group(1).strip()
    if len(name)>0:
        address = [name]
    else:
        address = []
    address.append(street)
    address.append(locality)
    
    address = ", ".join(address)
    
    car_access = ""
    for li in ul[3].find_all("li"):
        if li.span:
            if li.span.text == "Acces voiture":
                car_access = li.text.strip()

    return address, car_access

In [ ]:
def parse_day(res, game_date, day_i):
    events = []
    for j in range(len(res)):
        division = res[j]["d"]
        games = res[j]["m"]
        for game in games:
            event = {}
            if ":" in game["ds"]:
                time = game["ds"]
                home_team = game["ci"]
                away_team = game["co"]
                home_url_name = game["uti"]
                home_id = game["mi"]
                home_url = f"http://www.abssa.org/team/{home_id}/{home_url_name}"
                home_address, car_access = get_field_address(home_url)

                day, month, year = game_date.split("/")
                hour, minute = time.split(":")

                event["summary"] = f"ABSSA {division} DAY {day_i}: {home_team} - {away_team}"
                event["location"] = home_address
                event["start"] = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), tzinfo=pytz.timezone("Europe/Brussels"))
                event["end"] = event["start"]+datetime.timedelta(minutes=35*2+15)
                event["description"] = car_access
                
                event["home_team"] = home_team
                event["away_team"] = away_team
                event["division"] = division
                events.append(event)
    return events

In [ ]:
def get_tz():
    tzc = icalendar.Timezone()
    tzc.add('tzid', 'Europe/Brussels')
    tzc.add('x-lic-location', 'Europe/Brussels')

    tzs = icalendar.TimezoneStandard()
    tzs.add('tzname', 'CET')
    tzs.add('dtstart', datetime.datetime(1970, 10, 25, 3, 0, 0))
    tzs.add('rrule', {'freq': 'yearly', 'bymonth': 10, 'byday': '-1su'})
    tzs.add('TZOFFSETFROM', datetime.timedelta(hours=2))
    tzs.add('TZOFFSETTO', datetime.timedelta(hours=1))

    tzd = icalendar.TimezoneDaylight()
    tzd.add('tzname', 'CEST')
    tzd.add('dtstart', datetime.datetime(1970, 3, 29, 2, 0, 0))
    tzs.add('rrule', {'freq': 'yearly', 'bymonth': 3, 'byday': '-1su'})
    tzd.add('TZOFFSETFROM', datetime.timedelta(hours=1))
    tzd.add('TZOFFSETTO', datetime.timedelta(hours=2))

    tzc.add_component(tzs)
    tzc.add_component(tzd)
    return tzc
    

In [ ]:
def generate_cal(events, team):
    cal = Calendar()
    cal['X-WR-CALNAME'] = f'ABSSA {team}'
    cal['PRODID'] = str(uuid.uuid4())
    cal['VERSION'] = '0.1'
    cal.add_component(get_tz())
    for event in events:
        if team in (event["home_team"], event["away_team"]):
            ev = Event()
            ev.add('DTSTAMP', datetime.datetime.now())
            ev['uid'] = str(uuid.uuid4())
            
            ev.add('dtstart', event["start"])
            ev.add('dtend', event["end"])

            ev['location'] = vText(event["location"])
            ev['summary'] = vText(event["summary"])
            ev['description'] = vText(event["description"])
            cal.add_component(ev)
    return cal

In [ ]:
def url_safen(s):
    s = s.lower()
    s = s.replace(" ", "_")
    s = s.replace(".", "_")
    s = s.replace("'", "_")
    s = s.replace("-", "_")
    s = s.replace("(", "_")
    s = s.replace(")", "_")
    return s

In [ ]:
url = "http://www.abssa.org/championnat"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
ul = soup.find_all("ul", {"class": "dropdown-menu"})

In [ ]:
lis = ul[1].find_all("li")
game_days = []
game_days_value = []
for i in range(len(lis)):
    li = lis[i]
    game_days.append(li.text)
    game_days_value.append(li.a["data-value"])

In [ ]:
events = []
for i in range(len(game_days)):
    url = f"http://www.abssa.org/amatches/{game_days_value[i]}"
    res = requests.get(url)
    game_date = game_days[i]
    res = json.loads(res.text)
    
    day_event = parse_day(res, game_date, i+1)
    
    events.extend(day_event)

In [ ]:
divisions = set()
teams = {}
for e in events:
    divisions.add(e['division'])
    
for d in divisions:
    teams_div = set()
    for e in events:
        if d == e['division']:
            teams_div.add(e['home_team'])
            teams_div.add(e['away_team'])
        teams[d] = teams_div

In [ ]:
base_path = "../ics/2122"
season = "2122"
base_url = "http://icalx.com/public/simon123456/abssa_{season}_{divison}_{team}"

user = "simon123456"
pw = "z3Fz!YmFxtZE6xT"

write_disk = True
write_icalx = False

for d in divisions:
    for t in teams[d]:
        cal = generate_cal(events, t)
        t_safe = url_safen(t)
        
        if write_disk:
            cal_dir = os.path.join(base_path, str(d))
            cal_path = os.path.join(cal_dir, f"{t_safe}.ics")
            if not os.path.exists(cal_dir):
                os.makedirs(cal_dir)
            with open(cal_path, 'wb') as f:
                f.write(cal.to_ical())
            
        if write_icalx:
            cal_url = base_url.format(season=season, divison=d, team=t_safe)
            req = requests.put(cal_url, auth=(user, pw), data=cal.to_ical())
            print(cal_url)
            print(req.status_code)

In [ ]:
urljoin(base_url, str(d))

In [ ]:
icalx_url = "http://icalx.com/public/simon123456/tst"
user = "simon123456"
pw = "z3Fz!YmFxtZE6xT"
req = requests.put(icalx_url, auth=(user, pw), data=cal.to_ical())

In [ ]:
dir(req)

In [ ]:
tz = "BEGIN:VTIMEZONE TZID:Europe/Brussels X-LIC-LOCATION:Europe/Brussels BEGIN:DAYLIGHT TZOFFSETFROM:+0100 TZOFFSETTO:+0200 TZNAME:CEST DTSTART:19700329T020000 RRULE:FREQ=YEARLY;BYMONTH=3;BYDAY=-1SU END:DAYLIGHT BEGIN:STANDARD TZOFFSETFROM:+0200 TZOFFSETTO:+0100 TZNAME:CET DTSTART:19701025T030000 RRULE:FREQ=YEARLY;BYMONTH=10;BYDAY=-1SU END:STANDARD END:VTIMEZONE"

In [ ]:
print(cal.to_ical())

In [ ]:
cal.to_ical()

In [ ]:
cal.to_ical().decode().encode('latin-1')